In [1]:
import pandas as pd
import numpy as np
import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: '%.2f' % x)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from data_ingestion.ingest import get_data
from parameters.params import combined_data_file_path#, autogluon_params
from sklearn.model_selection import train_test_split
# from autogluon.tabular import TabularDataset, TabularPredictor
#from model_building.build_model import autogluon_model_build

In [2]:
# %matplotlib inline

# import numpy as np
# import warnings
# import matplotlib.pyplot as plt
# warnings.filterwarnings('ignore')
# np.random.seed(123)

In [3]:
# from autogluon.core.utils.loaders.load_pd import load
# train_data = load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sst/train.parquet')
# test_data = load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sst/dev.parquet')
# subsample_size = 1000  # subsample data for faster demo, try setting this to larger values
# train_data = train_data.sample(n=subsample_size, random_state=0)
# train_data.head(10)

In [4]:
data = get_data(combined_data_file_path)
data.head(2)

,author.properties.friends,author.properties.status_count,author.properties.verified,content.body,location.country,properties.platform,properties.sentiment,location.latitude,location.longitude
0,1689,22566.00,False,Can't believe I'm missing Love Island 😩,GB,twitter,1.00,51.57,0.46
1,114,1377.00,False,Last tweet about future wedding..... if I actually want a wedding I actually need to find a guy XD we all know I'm a loner. unlovable,GB,twitter,1.00,52.97,-1.17


In [5]:
data.rename(columns = {'author.properties.friends': 'friends',
                        'author.properties.status_count': 'status_count',
                        'author.properties.verified': 'verified', 
                        'content.body': 'text', 
                        'location.country': 'country',
                        'properties.platform': 'platform', 
                        'properties.sentiment': 'sentiment', 
                        'location.latitude': 'latitude',
                        'location.longitude': 'longitude'},
                        inplace=True)

In [6]:
data.columns

Index(['friends', 'status_count', 'verified', 'text', 'country', 'platform',
       'sentiment', 'latitude', 'longitude'],
      dtype='object')

In [7]:
df = data[['text', 'sentiment']]
df.head()

,text,sentiment
0,Can't believe I'm missing Love Island 😩,1.00
1,Last tweet about future wedding..... if I actually want a wedding I actually need to find a guy XD we all know I'm a loner. unlovable,1.00
2,"How many times does he wonna say the phrase ""i deal with shit"" #LoveIsland",-1.00
3,...even better if time travel were invented and I could go there back in 1876 and have a whiskey with Al #Deadwood,-1.00
4,@GreenAlty @ShakeyStephens My Mum in mid 80s pushed into road by Black migrants 2 sons with disabilities attacked robbed U THINK WE NEED MOR,0.00


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   text       2999 non-null   object 
 1   sentiment  2999 non-null   float64
dtypes: float64(1), object(1)
memory usage: 47.0+ KB


In [9]:
miss = df[df.isnull().any(axis=1)]
miss

,text,sentiment
1552,NaN,NaN


In [10]:
df = df.dropna(axis=0)

In [11]:
df.shape

(2999, 2)

In [12]:
len(df['text'].isnull())

2999

In [13]:
dfs = df.copy()

In [14]:
dfs = dfs[0:50]

In [15]:
train_data, test_data = train_test_split(
    dfs, test_size=0.1, random_state=42
)

In [16]:
#!pip install autogluon.text

In [ ]:
# from autogluon.text import TextPredictor

# predictor = TextPredictor(label='label', eval_metric='acc', path='./ag_sst')
# predictor.fit(train_data, time_limit=60)

from autogluon.text import TextPredictor

predictor = TextPredictor(label='sentiment', eval_metric='acc', path='artefacts/model')
predictor.fit(train_data, time_limit=240)



NOTE: Redirects are currently not supported in Windows or MacOs.
Global seed set to 123
